In [1]:
import pandas as pd
from collections import defaultdict
import utils
import hashlib
import pickle
import json
import sys
import glob

In [2]:
se_data_folder='auxiliary_data/semeval18_data'
se_test_data_folder='%s/test_data' % se_data_folder
se_trial_data_folder='%s/trial_data' % se_data_folder
se_frames_file='%s/gva_frames' % se_data_folder

input_data_folder='../data/input/full'
gold_data_folder='../data/gold/full/'

In [3]:
a_df = pd.read_pickle(se_frames_file)

In [4]:
len(a_df)

4580

In [5]:
exclude_empty_names=True

In [6]:
participant_data_per_document=defaultdict(dict)
identical_participants=defaultdict(set)
for index, row in a_df.iterrows():
    inc_uri=row['incident_uri']
    docs=list(row['hashed_ids'].values())
    cnt=1
    state=row['state']
    year=row['date'][-4:]
    for participant_data in row['participants']:
        inc_part_key='%s#%s' % (inc_uri, cnt)
        if exclude_empty_names and \
            ('Name' not in participant_data.keys() or not participant_data['Name'].strip()):
            continue
        if 'Status' in participant_data:
            status=participant_data['Status'].strip()
            if status=='Killed':
                participant_data['DeathPlace']=state
                participant_data['DeathDate']=year
        for doc_id in docs:
            string_to_encode='%s#%d#%s' % (doc_id, cnt, json.dumps(participant_data))
            new_id=hashlib.md5(string_to_encode.encode('utf-8')).hexdigest()
            participant_data_per_document[doc_id][new_id]=participant_data
            identical_participants[inc_part_key].add(new_id)
        cnt+=1


In [7]:
input_participants_file='%s/annotation/participants_input.p' % input_data_folder
gold_participants_file='%s/participants_gold.p' % gold_data_folder
out_gold_file = '%s/participants.json' % gold_data_folder

In [8]:
with open(input_participants_file, 'wb') as w:
    pickle.dump(participant_data_per_document, w)

In [9]:
with open(gold_participants_file, 'wb') as w:
    pickle.dump(identical_participants, w)

In [10]:
json_clusters=utils.transform_gold_to_json(identical_participants)

In [11]:
with open(out_gold_file, 'w') as w:
    json.dump(json_clusters, w)

#### 2a) Increase ambiguity by giving everyone the same first name

In [12]:
new_first_name = "John"

gva_samefirstname_annotations = '%s/annotation/participants_samefirstname.p' % input_data_folder

with open(input_participants_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info and participant_info['Name'].strip():
                old_name=participant_info['Name']
                first_name, *other_names = old_name.split()
                participant_info['Name']=' '.join([new_first_name] + other_names)
                print(old_name, participant_info["Name"])
    with open(gva_samefirstname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

 Haylee Garrison John Garrison
 Ryan Morales John Morales
John Morales John Morales
John Morales John Morales
 Marcus Cleveland Thomas Jr John Cleveland Thomas Jr
John Cleveland Thomas Jr John Cleveland Thomas Jr
John Cleveland Thomas Jr John Cleveland Thomas Jr
 Latina Herring John Herring
 Brendan Christian John Christian
 Branden Christian John Christian
 Bertis Gerard Herring Jr John Gerard Herring Jr
 Rakeya Jackson John Jackson
 Lazaro Paredesquelite John Paredesquelite
 Allen Dion Cashe John Dion Cashe
John Herring John Herring
John Christian John Christian
John Christian John Christian
John Gerard Herring Jr John Gerard Herring Jr
John Jackson John Jackson
John Paredesquelite John Paredesquelite
John Dion Cashe John Dion Cashe
 Benson Jones John Jones
John Jones John Jones
 Laila Goodwin John Goodwin
 Lashelle Goodwin John Goodwin
 Carlton Goodwin John Goodwin
John Goodwin John Goodwin
John Goodwin John Goodwin
John Goodwin John Goodwin
 Jacob Brockman John Brockman
 Landen Lav

John Janet Villegas-Hernandez John Janet Villegas-Hernandez
John Villegas-Hernandez John Villegas-Hernandez
John David Villegas-Hernandez John David Villegas-Hernandez
John Villegas John Villegas
John Villegas-Hernandez John Villegas-Hernandez
John Villegas-Hernandez John Villegas-Hernandez
John Janet Villegas-Hernandez John Janet Villegas-Hernandez
John Villegas-Hernandez John Villegas-Hernandez
John David Villegas-Hernandez John David Villegas-Hernandez
 Darnell Hardeman John Hardeman
 Timothy Harewood John Harewood
 Anthony Jaliel Lujan Hemmings-Taylor John Jaliel Lujan Hemmings-Taylor
 Jalecc Taylor John Taylor
John Jaliel Lujan Hemmings-Taylor John Jaliel Lujan Hemmings-Taylor
John Taylor John Taylor
 Iris Rodriguez John Rodriguez
 Frank Gomez John Gomez
 Jonathan Campos John Campos
 Manuel Watson John Watson
 Peter Louis Gonzalez John Louis Gonzalez
John Rodriguez John Rodriguez
John Gomez John Gomez
John Campos John Campos
John Watson John Watson
John Louis Gonzalez John Louis G

John Mata John Mata
John Huang John Huang
John Mata John Mata
 Corbin Wiederholt John Wiederholt
John Wiederholt John Wiederholt
John Wiederholt John Wiederholt
 David T Crowley John T Crowley
 Komel Crowley John Crowley
 Rani Crowley John Crowley
 Janessa Ramirez John Ramirez
 Jaimel Anderson John Anderson
 Dennis Eason John Eason
 Dejad Williams John Williams
 Tevin Sams John Sams
 Matthew Thomas John Thomas
 Warren Everette Thomas John Everette Thomas
 Jaxon Slaight John Slaight
 Joseph Slaight John Slaight
 Morgan Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
John Slaight John Slaight
 Raymond Gaines Jr. John Gaines Jr.
 Jaleel Taylor John Taylor
 Bill Thao John Thao
 Darmequaye Cohill John Cohill
 Samantha N. Carpenter John N. Carpenter
 Catherine L. Gessman John L. Gessman
 Brian Harleman John Harlem

John Easley John Easley
 Tyler Jordan Buchanan John Jordan Buchanan
 Calvin Dawayne Powers John Dawayne Powers
John Jordan Buchanan John Jordan Buchanan
John Dawayne Powers John Dawayne Powers
John Jordan Buchanan John Jordan Buchanan
John Dawayne Powers John Dawayne Powers
 Ruben Trinidad John Trinidad
John Trinidad John Trinidad
 Malik McNeese John McNeese
 Stevie Jefferson John Jefferson
 Tillie Turner John Turner
John McNeese John McNeese
John Jefferson John Jefferson
John Turner John Turner
 Anthony Frazier John Frazier
John Frazier John Frazier
 Roman Kellough John Kellough
John Kellough John Kellough
John Kellough John Kellough
 Trevon Johnson John Johnson
John Johnson John Johnson
 Suzanne Zaremba John Zaremba
 Charlotte Zaremba John Zaremba
 Sean Crizer John Crizer
John Zaremba John Zaremba
John Zaremba John Zaremba
John Crizer John Crizer
 Shamoya McKenzie John McKenzie
 Prince Scott John Scott
 David Hardy John Hardy
John McKenzie John McKenzie
John Scott John Scott
John Har

John Stover John Stover
John Gilliam John Gilliam
John Vallier John Vallier
John Stover John Stover
John Gilliam John Gilliam
John Vallier John Vallier
 Charles Ulery John Ulery
 John Bennett John Bennett
John Ulery John Ulery
John Bennett John Bennett
John Ulery John Ulery
John Bennett John Bennett
 David Bauspies John Bauspies
 Robert Sterling John Sterling
 Tyler Odell John Odell
 Brenda Marie Flues John Marie Flues
 Wayne Edmund Flues John Edmund Flues
 Maryann Nabil Rosail John Nabil Rosail
John Nabil Rosail John Nabil Rosail
 Roland Von Whitaker John Von Whitaker
 Rahsaan Smith John Smith
John Von Whitaker John Von Whitaker
John Smith John Smith
John Von Whitaker John Von Whitaker
John Smith John Smith
 Howard Smith John Smith
John Smith John Smith
 Carlos James John James
John James John James
 Bryant Silk John Silk
 Michael Harmon John Harmon
John Harmon John Harmon
 Joshua Biggs John Biggs
 Jalen Shaye Wilfong John Shaye Wilfong
 Jordan Masterson John Masterson
 Sarah Bonner J

...

 Emerson D. Ware John D. Ware
 Terrelle D. Russel John D. Russel
 Davon Ballard  John Ballard
 Michael Gaston John Gaston
 Devonte Dantzler John Dantzler
 Jamie Williams John Williams
 Sandy Gaddis Barnwell John Gaddis Barnwell
 Keith A Williams John A Williams
John Dantzler John Dantzler
John Williams John Williams
John Gaddis Barnwell John Gaddis Barnwell
John A Williams John A Williams
 Katherine Cooper John Cooper
 Christopher Martinez John Martinez
 Veronika Weiss John Weiss
 Elliot Rodger John Rodger
John Cooper John Cooper
John Martinez John Martinez
John Weiss John Weiss
John Rodger John Rodger
John Cooper John Cooper
John Martinez John Martinez
John Weiss John Weiss
John Rodger John Rodger
John Cooper John Cooper
John Martinez John Martinez
John Weiss John Weiss
John Rodger John Rodger
 Kenneth Mills John Mills
 German Booker John Booker
John Mills John Mills
John Booker John Booker
 Rolando Rojas Gonzalez John Rojas Gonzalez
John Rojas Gonzalez John Rojas Gonzalez
 Marcellus 

John Davis John Davis
John Carter John Carter
John Armstrong John Armstrong
John Collins John Collins
John Rhines John Rhines
John Wilson John Wilson
John T. Ward John T. Ward
 Ahmad M. Crudup John M. Crudup
 Dwayne Gene Dillard John Gene Dillard
 Shakim Lyons John Lyons
 Johnathan Taylor John Taylor
 Harry Herbet John Herbet
 James Coffield John Coffield
John Gene Dillard John Gene Dillard
John Lyons John Lyons
John Taylor John Taylor
John Herbet John Herbet
John Coffield John Coffield
 Tavin Yeakey John Yeakey
 Steven Eldridge John Eldridge
 Lonnie Christopher Roberson John Christopher Roberson
 Vincent Gordon John Gordon
 Vontarious Gordon John Gordon
John Christopher Roberson John Christopher Roberson
John Gordon John Gordon
John Gordon John Gordon
 Marques McCassie John McCassie
 Lorenze C DaVeiga John C DaVeiga
John McCassie John McCassie
John C DaVeiga John C DaVeiga
John McCassie John McCassie
John C DaVeiga John C DaVeiga
 Keith Warren John Warren
 Dennis Tindal John Tindal
 K

 Jaquan Graves John Graves
 Jamal Alexander John Alexander
 Roydricus McCray John McCray
 James Gill John Gill
 Roy Gill John Gill
 Steven Freeman John Freeman
John Gill John Gill
John Gill John Gill
John Freeman John Freeman
 O'Neil Bandoo John Bandoo
 Errol Hillary John Hillary
John Bandoo John Bandoo
John Hillary John Hillary
John Bandoo John Bandoo
John Hillary John Hillary
John Bandoo John Bandoo
John Hillary John Hillary
 Daisy Joins John Joins
John Joins John Joins
John Joins John Joins
 Tammy Lynette Pearce John Lynette Pearce
 Selby Gene Outland John Gene Outland
 Paul Shane Pearce John Shane Pearce
 Dominique Nicole Privette John Nicole Privette
John Lynette Pearce John Lynette Pearce
John Gene Outland John Gene Outland
John Shane Pearce John Shane Pearce
John Nicole Privette John Nicole Privette
 Linda Hall John Hall
 Shacora Jackson John Jackson
 Elijah Jackson John Jackson
 Scott Travis Thompson John Travis Thompson
 Nateyah Yafah Hines John Yafah Hines
 Lionel Parks John 

John Rogers John Rogers
 Gerald M. Berkey John M. Berkey
 Jackson L. Edens John L. Edens
 Terron R. McGrath John R. McGrath
 Dugan Lawton John Lawton
 Enrique “Ricky” Lamere John “Ricky” Lamere
John M. Berkey John M. Berkey
John L. Edens John L. Edens
John R. McGrath John R. McGrath
John Lawton John Lawton
John “Ricky” Lamere John “Ricky” Lamere
John M. Berkey John M. Berkey
John L. Edens John L. Edens
John R. McGrath John R. McGrath
John Lawton John Lawton
John “Ricky” Lamere John “Ricky” Lamere
John M. Berkey John M. Berkey
John L. Edens John L. Edens
John R. McGrath John R. McGrath
John Lawton John Lawton
John “Ricky” Lamere John “Ricky” Lamere
John M. Berkey John M. Berkey
John L. Edens John L. Edens
John R. McGrath John R. McGrath
John Lawton John Lawton
John “Ricky” Lamere John “Ricky” Lamere
 Monte Compton John Compton
 Donte Jefferson John Jefferson
 Eric Lee Anthony John Lee Anthony
John Compton John Compton
John Jefferson John Jefferson
John Lee Anthony John Lee Anthony
John 

John John Wilson John John Wilson
John Augustine Garza III John Augustine Garza III
John Linscomb John Linscomb
John Constable Danny Luna John Constable Danny Luna
John Constable Jaime Ayala John Constable Jaime Ayala
John Slaughter John Slaughter
John John Wilson John John Wilson
John Augustine Garza III John Augustine Garza III
John Linscomb John Linscomb
John Constable Danny Luna John Constable Danny Luna
John Constable Jaime Ayala John Constable Jaime Ayala
John Slaughter John Slaughter
John John Wilson John John Wilson
John Augustine Garza III John Augustine Garza III
John Linscomb John Linscomb
John Constable Danny Luna John Constable Danny Luna
John Constable Jaime Ayala John Constable Jaime Ayala
John Slaughter John Slaughter
John John Wilson John John Wilson
John Augustine Garza III John Augustine Garza III
 Steven Burks John Burks
 Pierre Devlin John Devlin
John Burks John Burks
John Devlin John Devlin
 Michael Shaw John Shaw
 Michael Myers Jr. John Myers Jr.
 Dameisha Clark 

#### 2b) Increase ambiguity by giving everyone the same last name

In [13]:
new_last_name = "Smith"

gva_samelastname_annotations = '%s/annotation/participants_samelastname.p' % input_data_folder

with open(input_participants_file, 'rb') as f:
    gva_input_annotations=pickle.load(f)
    for doc_id, data in gva_input_annotations.items():
        for participant_id, participant_info in data.items():
            if 'Name' in participant_info and participant_info['Name'].strip():
                old_name=participant_info['Name']
                *other_names, last_name = old_name.split()
                participant_info['Name']=' '.join(other_names + [new_last_name])
    with open(gva_samelastname_annotations, 'wb') as w:
        pickle.dump(gva_input_annotations, w)

### 3. Gather all documents in JSON format from the violent corpus

In [14]:
corpus_location='../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus'

In [15]:
documents_to_store=set(participant_data_per_document.keys())

In [17]:
found_documents=set()
for fn in glob.glob('%s/*/*.json' % corpus_location):
    file_basename=fn.split('/')[-1]
    doc_name=file_basename.split('.')[0]
    print(doc_name, doc_name in documents_to_store)
    if doc_name in documents_to_store and doc_name not in found_documents:
        with open(fn, 'rb') as pickfile:
            a_news_item=pickle.load(pickfile)
        new_json={'title': a_news_item.title, 
                  'content': a_news_item.content, 
                  'dct': a_news_item.dct.strftime("%Y-%m-%d")}
        with open('%s/text/%s' % (input_data_folder, file_basename), 'w') as target_json:
            json.dump(new_json, target_json)
        
        found_documents.add(doc_name)
        print('copied %s' % fn)

f739daaab4ddad7ca8bffab7e8befac3 False
ff81f3629d65117d9309aab23cfc7821 False
da90c693f7ecfe131d2cb843f7257e1d False
f256b3994e839af90fdb3864cd7cc970 False
e20fc3c203ee1479c5a910076a4885ad False
361004dfe9372fdfe533487fcc8fd521 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/757001/361004dfe9372fdfe533487fcc8fd521.json
f6c2393864253a458c036d45b46c333b True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/573759/f6c2393864253a458c036d45b46c333b.json
5e2fd20f15d6b988be4a50ac33c9111f True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/573759/5e2fd20f15d6b988be4a50ac33c9111f.json
2d1838df7aa757ec6860d07e8a01664f True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/369196/2d1838df7aa757ec6860d07e8a01664f.json
92c573dc5ba699dc9aa2c21dc2c79c58 True
copied ../../../../SemEval/LongTailQATask/EventRegi

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/571835/068ae0aa2886b759ea8968ecebbb38f3.json
ed22bfd9e58c965f82241abd99024236 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/286810/ed22bfd9e58c965f82241abd99024236.json
871b3d31fb0284da5e90c25779e20974 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/286810/871b3d31fb0284da5e90c25779e20974.json
f30406686618d8f117b360db087a8794 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/286810/f30406686618d8f117b360db087a8794.json
fbbe063e1e99063e573507872652e16e True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/486977/fbbe063e1e99063e573507872652e16e.json
8b3f8ab0e873c8362e0b1b4d3860f47e False
40c23df755918af9a84141255911ff3d True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViole

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/741536/98fbe546b7d9e1b09ecf1578fca96d97.json
bb04045425bf331914ac1349c2681778 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/744044/bb04045425bf331914ac1349c2681778.json
68bac46b8749787f3eb1c7e6d5381dcd True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/744044/68bac46b8749787f3eb1c7e6d5381dcd.json
d04683209256dab4593d21036b7cdbec True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/744044/d04683209256dab4593d21036b7cdbec.json
0277918340e5b5be1200adf113096ec8 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/504259/0277918340e5b5be1200adf113096ec8.json
a796cf28665991553d9b1135b93e9a94 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/504259/a7

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/269072/9c88b6c4e17ef1e6db9341518caed7b3.json
2905005f0be44d57559089d18654c4b4 False
02b8f226e8f9ad8735162e60c0ae57c2 False
8f838d331fb63405949e5cd7f0a9f2fd False
8b6d57873119d177f08c6c6f9a5078b9 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/376920/8b6d57873119d177f08c6c6f9a5078b9.json
8f06ee41a22b7b2a65e10d1ac2b626dd False
7d5587c797fb8a0859cefbd987d2547c True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/798236/7d5587c797fb8a0859cefbd987d2547c.json
d377f5d8960a18acc6384d2b43432c8b False
1e220eac93d180b53592f72f3d4a220c True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/773666/1e220eac93d180b53592f72f3d4a220c.json
bd8100cf09776f6f3b23400af1df4cec True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/757090/4124734ec7bcf04940c3f17d8cf2c4d6.json
5d3769385c973306ff63d3ffd17bce61 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/757090/5d3769385c973306ff63d3ffd17bce61.json
de32bb33bc4b4e0b109a87e921295d38 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/757090/de32bb33bc4b4e0b109a87e921295d38.json
bfbf30838e90ee0ae7e73317b948e765 False
e85d29edcb19834504a68ab702e48670 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/762377/e85d29edcb19834504a68ab702e48670.json
96163728bb02a60f0b4d3540cfb24616 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/751367/96163728bb02a60f0b4d3540cfb24616.json
a59a00470489941c2118c20f63ce5211 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViole

...

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/625724/90ea9fe3600e7a60f7013fea3c4e9da5.json
3ae4c931eaa78b70f707164faad83b3c True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/813313/3ae4c931eaa78b70f707164faad83b3c.json
c56b6891c8c0d4e3bb9c900d3bab21f3 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/813313/c56b6891c8c0d4e3bb9c900d3bab21f3.json
87d95894c7f39c7a897858e942a4b31f True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/491181/87d95894c7f39c7a897858e942a4b31f.json
4b02549ec8222a842ef43db908d588b9 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/491181/4b02549ec8222a842ef43db908d588b9.json
2adcc9186fe7b0852348ae29a431c10f True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/491181/2a

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/349976/dc17323ba73218b5ef6cfe797658bf9f.json
48acbde54454ab4ee540a2eee20d6357 False
e74a73e0ffdd3e34bd0c8e2b05766c4c False
d28282604f7104eed44ccaaabf26f493 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/588918/d28282604f7104eed44ccaaabf26f493.json
9543bb8549c4be82515654d1411639ae False
0c02438c6829d2191b3d1252fd3805f8 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/424335/0c02438c6829d2191b3d1252fd3805f8.json
1403eb502b1bdc566f4a6b884b28bf6d True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/749184/1403eb502b1bdc566f4a6b884b28bf6d.json
1bf7d14238d0d8989aeab5deaa9be8a1 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/749184/1bf7d14238d0d8989aeab5deaa9be8a1.json
25f56f4022012e43b704930bb7

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/721895/9f1b8d3c44cc1677660a751409c8bf99.json
251ee2f58dc9fe8fc32e6f9394ee59fb True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/556436/251ee2f58dc9fe8fc32e6f9394ee59fb.json
ee5aa38707066467cbf4ff488d8362bf True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/556436/ee5aa38707066467cbf4ff488d8362bf.json
ea689b78ca8ff3ed11f667b9e4f13dfb False
b41ce2ba48f97cb5aaaded2ce31a513b False
2fb19095f618bfbd349c310f8995c4dc True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/419826/2fb19095f618bfbd349c310f8995c4dc.json
65748d48987f53eeaff66570bc6da0db True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/419826/65748d48987f53eeaff66570bc6da0db.json
64f3f7fd801a1069a09a90180c2d39fb True
copied ../../../../SemEval/

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/479561/bcd1890572a4c27b3da866cbfac38649.json
f5898f5270173ba0aae5d7986460145d True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/479561/f5898f5270173ba0aae5d7986460145d.json
1467e1e271f824f2a38dc6bda69250ef False
35e2d10d9ca9e1f92fa85b0b3368aff3 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/765323/35e2d10d9ca9e1f92fa85b0b3368aff3.json
31df08d95042838a39ff1887ed5b041f False
01bf31f468199591907bdf1a33932ad3 False
9eacd854d1bf2e32bcdf415b02a62eeb False
e7e41e7715d246c4dff677e3675c5917 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/299678/e7e41e7715d246c4dff677e3675c5917.json
708423791b8e807eb0ab4d40a77e3743 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/647101/708423791b8e807eb0ab4d40a

copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/794285/a85f41b3ab23fa8a9dce54dc6f888249.json
17671bd4d2e6975d9bae749590ca2523 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/794285/17671bd4d2e6975d9bae749590ca2523.json
75c66484bfbd0f0fe6bb1cd02f8e7306 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/794285/75c66484bfbd0f0fe6bb1cd02f8e7306.json
d8b8d48394f1f597541a3fbebacdf08a True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/738576/d8b8d48394f1f597541a3fbebacdf08a.json
1785e402f476eac6587bbceb80a198da False
42212d43d66989545dc772c225172497 False
deb4d88ba27b865b60809cc2da91c9d5 True
copied ../../../../SemEval/LongTailQATask/EventRegistries/GunViolenceArchive/the_violent_corpus/314316/deb4d88ba27b865b60809cc2da91c9d5.json
27bf41ee90a86b1ae55427b4c0c0883a False
08b9eb842499f8e6fae1629a8d

In [18]:
len(found_documents)

4479